In [ ]:
### !!! AFTER DIMENSIONALITY REDUCTION !!! ###

import pandas as pd
import numpy as np
import os

In [ ]:
import instructions

path = os.path.split(os.getcwd())
data_directory = os.path.join(path[0], 'data\\raw')
#data_directory = os.path.join(os.getcwd(), 'data\\raw')


X = instructions.load_csv(data_directory, 'train_data')
y = instructions.load_csv(data_directory, 'train_labels')

In [ ]:

from sklearn.pipeline import Pipeline

pipe = Pipeline(steps=[("classifier", LogisticRegression())])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

search_space = [{"classifier" : [LogisticRegression()],
                "classifier__penalty" : ['l1', 'l2'],
                "classifier__C" : [np.logspace(0, 4, 10)]},
                {"classifier" : [RandomForestClassifier()],
                "classifier__n_estimators" : [10, 100, 1000],
                "classifier__max_features" : [1, 2, 3]},
                {"classifier" : [KNeighborsClassifier()],
                "classifier__n_neighbors" : [3, 5, 7, 9]}]

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

splits = 3
n_repeats = 10
rskf = RepeatedStratifiedKFold(n_splits=splits,n_repeats=n_repeats)

multi_grid_search = GridSearchCV(pipe, search_space, cv=5, verbose=3)   

In [ ]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_std = std.fit_transform(X)
X_std

In [ ]:
for fold_nb, (train_index, test_index) in enumerate(rskf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train_std = std.fit_transform(X_train)
    multi_grid_search.fit(X_train_std, y_train)
    print(fold_nb)

In [ ]:
multi_grid_search.best_estimator_, multi_grid_search.best_params_, multi_grid_search.best_score_